# SNN that detects High Frequency Oscillations (HFOs) with constant parameters
This notebook is a simple example of how to use a Spiking Neural Network (SNN) to detect HFOs

### What is an HFO?
High Frequency Oscillations (HFOs) are a type of brain activity that occurs in the range of 80-500 Hz. They are believed to be related to the generation of seizures in patients with epilepsy. The detection of HFOs is an important task in the diagnosis and treatment of epilepsy. 

In terms of electrophysiology, HFOs are characterized by their high frequency and short duration, often lasting only a few milliseconds. The wave of a typical HFO consists of at least 4 UP and DOWN waves.

In [1]:
from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense
import numpy as np

LIF?

Init signature: LIF(*args, **kwargs)
Docstring:     
Leaky-Integrate-and-Fire (LIF) neural Process.

LIF dynamics abstracts to:
u[t] = u[t-1] * (1-du) + a_in         # neuron current
v[t] = v[t-1] * (1-dv) + u[t] + bias  # neuron voltage
s_out = v[t] > vth                    # spike if threshold is exceeded
v[t] = 0                              # reset at spike

Parameters
----------
shape : tuple(int)
    Number and topology of LIF neurons.
u : float, list, numpy.ndarray, optional
    Initial value of the neurons' current.
v : float, list, numpy.ndarray, optional
    Initial value of the neurons' voltage (membrane potential).
du : float, optional
    Inverse of decay time-constant for current decay. Currently, only a
    single decay can be set for the entire population of neurons.
dv : float, optional
    Inverse of decay time-constant for voltage decay. Currently, only a
    single decay can be set for the entire population of neurons.
bias_mant : float, list, numpy.ndarray, optiona

### Check WD (change if necessary) and file loading

In [2]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/snn" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/snn"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis
File Location:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/snn
New Working Directory:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/snn


## Create the Custom Input Layer

### Define function to read the input data from the csv file and generate the corresponding spike events

In [3]:
import pandas as pd

def read_spike_events(file_path: str):
    """Reads the spike events from the input file and returns them as a numpy array

    Args:
        file_path (str): name of the file containing the spike events
    """
    spike_events = []

    try:
        # Read the spike events from the file
        df = pd.read_csv(file_path, header=None)

        # Detect errors
        if df.empty:
            raise Exception("The input file is empty")

        # Convert the scientific notation values to integers if any exist
        df = df.applymap(lambda x: int(float(x)) if (isinstance(x, str) and 'e' in x) else x)

        # Convert the dataframe to a numpy array
        spike_events = df.to_numpy()
        return spike_events[0]
    except Exception as e:
        print("Unable to read the input file: ", file_path, " error:", e)

    return spike_events

### Load the UP and DOWN spikes from the CSV Files

In [4]:
from utils.input import read_spike_events, MarkerType, band_to_file_name
from utils.io import preview_np_array

# Define the path of the files containing the spike events
INPUT_PATH = "../signal_to_spike/results/custom_subset_90-119_segment500_200"

# Declare if using ripples, fast ripples, or both
chosen_band = MarkerType.FAST_RIPPLE     # RIPPLE, FAST_RIPPLE, or BOTH
band_file_name = band_to_file_name(chosen_band)

# Call the function to read the spike events
up_spikes_file_path = f"{INPUT_PATH}/{band_file_name}_up_spike_train_3.csv"
up_spike_train = read_spike_events(up_spikes_file_path)

down_spikes_file_path = f"{INPUT_PATH}/{band_file_name}_down_spike_train_-3.csv"
down_spike_train = read_spike_events(down_spikes_file_path)

preview_np_array(up_spike_train, "Spike Events")
preview_np_array(down_spike_train, "Spike Events")

Spike Events Shape: (2799, 2).
Preview: [[ 1.00195312e+03 -1.00000000e+00]
 [ 1.00244141e+03 -1.00000000e+00]
 [ 1.00537109e+03 -1.00000000e+00]
 [ 1.00585938e+03 -1.00000000e+00]
 [ 1.00830078e+03 -1.00000000e+00]
 ...
 [ 1.19008301e+05 -1.00000000e+00]
 [ 1.19008789e+05 -1.00000000e+00]
 [ 1.19012207e+05 -1.00000000e+00]
 [ 1.19015137e+05 -1.00000000e+00]
 [ 1.19015625e+05 -1.00000000e+00]]
Spike Events Shape: (2735, 2).
Preview: [[ 1.00097656e+03 -1.00000000e+00]
 [ 1.00390625e+03 -1.00000000e+00]
 [ 1.00683594e+03 -1.00000000e+00]
 [ 1.00732422e+03 -1.00000000e+00]
 [ 1.01074219e+03 -1.00000000e+00]
 ...
 [ 1.19006836e+05 -1.00000000e+00]
 [ 1.19010254e+05 -1.00000000e+00]
 [ 1.19013672e+05 -1.00000000e+00]
 [ 1.19014160e+05 -1.00000000e+00]
 [ 1.19016602e+05 -1.00000000e+00]]


### Define the SpikeEvent Generator Interface

In [5]:
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import OutPort

class SpikeEventGen(AbstractProcess):
    """Input Process that generates spike events based on the input file

    Args:
        @out_shape (tuple): Shape of the output port
        @exc_spike_events (np.ndarray): Excitatory spike events
        @inh_spike_event (np.ndarray): Inhibitory spike events
        @name (str): Name of the process
    """
    def __init__(self, out_shape: tuple, exc_spike_events: np.ndarray, inh_spike_event: np.ndarray, name: str) -> None:
        super().__init__(name=name)
        self.s_out = OutPort(shape=out_shape)
        self.exc_spike_events = Var(shape=exc_spike_events.shape, init=exc_spike_events)
        self.inh_spike_events = Var(shape=inh_spike_event.shape, init=inh_spike_event)


## Define the Architecture of the Network

In [6]:
# Define the number of neurons in the Input Spike Event Generator
n_spike_gen = 2  # 2 neurons in the input spike event generator

# Define the number of neurons in each LIF Layer
n_lif1 = 256   # 256 neurons in the first LIF layer
# n2 = 1  # 1 neuron in the second layer

### Choose the LIF Models to use

In [7]:
use_refractory = False

### Define the LIF parameters

In [8]:
# Constants for the LIF Process
v_th = 1
v_init = 0

# LIF1 Process
dv1 = 0.07
du1 = 0.2  

### Create the LIF Processes

In [9]:
if not use_refractory:
    # Create LIF1 process
    lif1 = LIF(shape=(n_lif1,),  # There are 2 neurons
            vth=v_th,  # TODO: Verify these initial values
            v=v_init,
            dv=dv1,    # Inverse of decay time-constant for voltage decay
            du=du1,  # Inverse of decay time-constant for current decay
            bias_mant=0,
            bias_exp=0,
            name="lif1")

### Create the Refractory LIF Processes

In [10]:
from lava.proc.lif.process import LIFRefractory
from lava.magma.core.process.process import LogConfig
import logging

# Constants for the Refractory LIF Process
refrac_period = 20   # Number of time-steps for the refractory period

if use_refractory:
    # Create Refractory LIF1 process
    lif1 = LIFRefractory(shape=(n_lif1,),  # There are 2 neurons
            vth=v_th,  # TODO: Verify these initial values
            v=v_init,
            dv=dv1,    # Inverse of decay time-constant for voltage decay
            du=du1,  # Inverse of decay time-constant for current decay
            bias_mant=0,
            bias_exp=0,
            refractory_period=refrac_period,
            name="lif1",
            # log_config=LogConfig(level=logging.DEBUG, level_console=logging.DEBUG, logs_to_file=False)
            )

### Create a `ConfigTimeConstantsLIF` object

#### Define the time constants for the `ConfigTimeConstantsLIF` neurons
The synapse time constants, corresponding to `du_exc` and `du_inh` will be different for each neuron in the LIF layer. Likewise, the excitatory and inhibitory time constants will also differ for each neuron in order to capture the dynamics of the HFOs.

In [11]:
from utils.neuron_dynamics import time_constant_to_fraction
# Create the np arrays for the time constants of each neuron

# For Fast-Ripples, the time constants of each neuron are drawn randomly from a normal distribution with an IQR of [0.3ms, 2.7ms]
fr_IQR = [0.3, 2.7] # Inter-Quartile Range for the time constants of the Fast-Ripple neurons
fr_mu = np.mean(fr_IQR)  # Midpoint of the IQR

# Calculate the standard deviation of the normal distribution
# For a normal distribution, the first quartile is ~0.675 standard deviations below the mean
fr_std_dev = (fr_IQR[1] - fr_IQR[0]) / (2 * 0.675)  # standard deviation is the IQR divided by 2*0.675

print("fr_IQR: ", fr_IQR)

fr_IQR:  [0.3, 2.7]


### Generate a random distribution of the Synaptic Excitatory time constants 

In [12]:
# Generate the time constants for the Fast-Ripple neurons
exc_syn_time_constants = np.random.normal(fr_mu, fr_std_dev, n_lif1)
# preview_np_array(exc_syn_time_constants, "exc_syn_time_constants", edge_items=10)
print("Min and max time constants before:", np.min(exc_syn_time_constants), np.max(exc_syn_time_constants))


# Cannot have negative time constants. Make them 0 or positive?
# exc_syn_time_constants = np.clip(exc_syn_time_constants, a_min=0, a_max=None)
exc_syn_time_constants = np.abs(exc_syn_time_constants)
print("Min and max time constants after:", np.min(exc_syn_time_constants), np.max(exc_syn_time_constants))

Min and max time constants before: -3.9663194912690267 6.7583473203457975
Min and max time constants after: 0.01654983386995479 6.7583473203457975


### The inhibitory time constants will be calculated from the excitatory time constants by subtracting a value in a range

In [13]:
# Generate the inhibitory time constants by subtracting a random value in a range from the excitatory time constants

# Generate the random values to subtract from the excitatory time constants
inh_syn_offset = np.random.uniform(0.1, 1.0, n_lif1)

# Subtract the random values from the excitatory time constants to get the inhibitory time constants
inh_syn_time_constants = exc_syn_time_constants - inh_syn_offset

# Clip the inhibitory time constants to be positive or equal to the minimum found excitatory time constant
inh_syn_time_constants = np.clip(inh_syn_time_constants, a_min=np.min(exc_syn_time_constants), a_max=None)

In [14]:
# Convert the excitatory time constants to fractions (du_exc values) that are used in the LAVA Processes dynamics
exc_syn_time_constants_frac = time_constant_to_fraction(exc_syn_time_constants)
preview_np_array(exc_syn_time_constants_frac, "exc_syn_time_constants_frac", edge_items=5)
print(np.min(exc_syn_time_constants_frac), np.max(exc_syn_time_constants_frac))


exc_syn_time_constants_frac Shape: (256,).
Preview: [0.51514806 0.27494502 0.96534805 0.16233966 0.1758811  ... 0.43489878
 0.39497254 0.4424901  0.68232611 0.37742878]
0.13753884476074674 1.0


In [15]:
# Convert the inhibitory time constants to fractions (du_inh values) that are used in the LAVA Processes dynamics
inh_syn_time_constants_frac = time_constant_to_fraction(inh_syn_time_constants)
preview_np_array(inh_syn_time_constants_frac, "inh_syn_time_constants_frac", edge_items=5)
print(np.min(inh_syn_time_constants_frac), np.max(inh_syn_time_constants_frac))


inh_syn_time_constants_frac Shape: (256,).
Preview: [0.73489657 0.30989497 1.         0.18911483 0.18344243 ... 0.46433055
 0.4745407  0.47455724 0.96055771 0.53804906]
0.14863618446497173 1.0


In [16]:
from lava.proc.lif.process import ConfigTimeConstantsLIF

configLIF = ConfigTimeConstantsLIF(shape=(n_lif1,),  # There are 2 neurons
            vth=v_th,  # TODO: Verify these initial values
            v=v_init,
            dv=dv1,    # Inverse of decay time-constant for voltage decay
            du_exc=exc_syn_time_constants_frac,  # Inverse of decay time-constant for excitatory current decay
            du_inh=inh_syn_time_constants_frac,  # Inverse of decay time-constant for inhibitory current decay
            bias_mant=0,
            bias_exp=0,
            name="lif1")

configLIF.du_exc

dv is scalar, converting to numpy array


Variable: du_exc
    shape: (256,)
    init: [0.51514806 0.27494502 0.96534805 0.16233966 0.1758811  ... 0.43489878
 0.39497254 0.4424901  0.68232611 0.37742878]
    shareable: True
    value: [0.51514806 0.27494502 0.96534805 0.16233966 0.1758811  ... 0.43489878
 0.39497254 0.4424901  0.68232611 0.37742878]

### Create the Dense Layers

In [17]:
# Create Dense Process to connect the input layer and LIF1
# create weights of the dense layer
# dense_weights_input = np.eye(N=n1, M=n1)
# Fully Connected Layer from n_spike_gen neurons to n_lif1 neurons
dense_weights_input = np.ones(shape=(n_lif1, n_spike_gen))

# Make the weights (synapses) connecting the odd-parity neurons of the input layer to the network negative (inhibitory)
dense_weights_input[:, 1::2] *= -1

# multiply the weights of the Dense layer by a constant
weights_scale_input = 0.3
dense_weights_input *= weights_scale_input
dense_input = Dense(weights=np.array(dense_weights_input), name="DenseInput")

#### Look at the weights of the Dense Layers

In [18]:
# Weights of the Input Dense Layer
dense_input.weights.get()

array([[ 0.3, -0.3],
       [ 0.3, -0.3],
       [ 0.3, -0.3],
       [ 0.3, -0.3],
       [ 0.3, -0.3],
       ...,
       [ 0.3, -0.3],
       [ 0.3, -0.3],
       [ 0.3, -0.3],
       [ 0.3, -0.3],
       [ 0.3, -0.3]])

### Map the input channels to the corresponding indexes in the input layer
Since the input channels in the input file may be of any number, we need to **map the input channels to the corresponding indexes in the input layer**. This is done by the `channel_map` dictionaries.

The network expects an UP and DOWN spike train for each channel. Thusly, let's define 2 dictionaries, one for the UP spikes and one for the DOWN spikes. We want the UP and DOWN spike trains to be followed by each other in the input layer for each channel.

In [19]:
# Map the channels of the input file to the respective index in the output list of SpikeEventGen

# Define the mapping of the channels of the UP spike train to the respective index in the output list of SpikeEventGen
up_channel_map = {-1: 0}
# Define the mapping of the channels of the DOWN spike train to the respective index in the output list of SpikeEventGen
down_channel_map = {-1: 1}

## Define constants related to the simulation time

In [20]:
init_offset = 1000  # 33400      #   
virtual_time_step_interval = 1  # TODO: Check if this should be the time-step value. it is not aligned with the sampling rate of the input data

num_steps = 300    # 200 # Number of steps to run the simulation

### Update the UP and DOWN spike trains to include only the spikes that occur within the simulation time

In [21]:
# Iterate the UP spike train to find the spikes that occur within the time interval
up_train_start = -1
up_train_end = 0
for i, (spike_time, _) in enumerate(up_spike_train):
    if up_train_start == -1 and spike_time >= init_offset:
        up_train_start = i
    
    if spike_time > init_offset + num_steps:
        up_train_end = i
        break

# Slice the spike train to the time interval
up_spike_train_interval = []
if up_train_start != -1:
    # If there are spikes in the interval
    up_spike_train_interval = up_spike_train[up_train_start:up_train_end]
    
preview_np_array(up_spike_train_interval, "Spike Events")

Spike Events Shape: (11, 2).
Preview: [[ 1.00195312e+03 -1.00000000e+00]
 [ 1.00244141e+03 -1.00000000e+00]
 [ 1.00537109e+03 -1.00000000e+00]
 [ 1.00585938e+03 -1.00000000e+00]
 [ 1.00830078e+03 -1.00000000e+00]
 ...
 [ 1.01269531e+03 -1.00000000e+00]
 [ 1.01562500e+03 -1.00000000e+00]
 [ 1.01611328e+03 -1.00000000e+00]
 [ 1.02050781e+03 -1.00000000e+00]
 [ 1.02441406e+03 -1.00000000e+00]]


In [22]:
# Iterate the UP spike train to find the spikes that occur within the time interval
down_train_start = -1
down_train_end = 0
for i, (spike_time, _) in enumerate(down_spike_train):
    if down_train_start == -1 and spike_time >= init_offset:
        down_train_start = i
    
    if spike_time > init_offset + num_steps:
        down_train_end = i
        break

# Slice the spike train to the time interval
down_spike_train_interval = []
if down_train_start != -1:
    # If there are spikes in the interval
    down_spike_train_interval = down_spike_train[down_train_start:down_train_end]
    
preview_np_array(down_spike_train_interval, "Spike Events")

Spike Events Shape: (10, 2).
Preview: [[ 1.00097656e+03 -1.00000000e+00]
 [ 1.00390625e+03 -1.00000000e+00]
 [ 1.00683594e+03 -1.00000000e+00]
 [ 1.00732422e+03 -1.00000000e+00]
 [ 1.01074219e+03 -1.00000000e+00]
 [ 1.01416016e+03 -1.00000000e+00]
 [ 1.01464844e+03 -1.00000000e+00]
 [ 1.01757812e+03 -1.00000000e+00]
 [ 1.02148438e+03 -1.00000000e+00]
 [ 1.02539062e+03 -1.00000000e+00]]


## Implement the `SpikeEventGenerator` Model

In [23]:
from lava.magma.core.model.py.model import PyLoihiProcessModel  # Processes running on CPU inherit from this class
from lava.magma.core.resources import CPU
from lava.magma.core.decorator import implements, requires
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.model.py.ports import PyOutPort

@implements(proc=SpikeEventGen, protocol=LoihiProtocol)
@requires(CPU)
class PySpikeEventGenModel(PyLoihiProcessModel):
    """Spike Event Generator Process implementation running on CPU (Python)
    Args:
    """
    s_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, float)   # IT IS POSSIBLE TO SEND FLOATS AFTER ALL
    exc_spike_events: np.ndarray = LavaPyType(np.ndarray, np.ndarray)
    inh_spike_events: np.ndarray = LavaPyType(np.ndarray, np.ndarray)

    def __init__(self, proc_params) -> None:
        super().__init__(proc_params=proc_params)
        # print("spike events", self.spike_events.__str__())    # TODO: Check why during initialization the variable prints the class, while during run it prints the value
        
        self.curr_exc_idx = 0     # Index of the next excitatory spiking event to send
        self.curr_inh_idx = 0     # Index of the next inhibitory spiking event to send
        self.virtual_time_step_interval = virtual_time_step_interval  # 1000    # Arbitrary time between time steps (in microseconds). This is not a real time interval (1000ms = 1s)
        self.init_offset = init_offset        # 698995               # Arbitrary offset to start the simulation (in microseconds)
        
        # Try to increment the curr_exc_idx and curr_inh_idx to the first spike event that is greater than the init_offset here?

    def run_spk(self) -> None:
        spike_data = np.zeros(self.s_out.shape) # Initialize the spike data to 0
        
        #print("time step:", self.time_step)

        # If the current simulation time is greater than a spike event, send a spike in the corresponding channel
        currTime = self.init_offset + self.time_step*self.virtual_time_step_interval

        spiking_channels = set()   # List of channels that will spike in the current time step

        # Add the excitatory spike events to the spike_date
        while (self.curr_exc_idx < len(self.exc_spike_events)) and currTime >= self.exc_spike_events[self.curr_exc_idx][0]:
            # Get the channel of the current spike event
            curr_channel = self.exc_spike_events[self.curr_exc_idx][1]

            # Check if the channel is valid (belongs to a channel in the up_channel_map therefore it has an output index)
            if curr_channel not in up_channel_map:
                self.curr_exc_idx += 1
                continue    # Skip the current spike event

            # Check if the next spike belongs to a channel that will already spike in this time step
            # If so, we don't add the event and stop looking for more events
            if curr_channel in spiking_channels:
                break

            # Add the channel to the list of spiking channels
            spiking_channels.add(curr_channel)

            # Get the output index of the current channel according to the up_channel_map
            out_idx = up_channel_map[curr_channel]
            if out_idx < self.s_out.shape[0]:   # Check if the channel is valid
                # Update the spike_data with the excitatory spike event (value = 1.0)
                spike_data[out_idx] = 1.0   # Send spike (value corresponds to the punctual current of the spike event)

            # Move to the next spike event
            self.curr_exc_idx += 1

        # Add the inhibitory spike events to the spike_date
        while (self.curr_inh_idx < len(self.inh_spike_events)) and currTime >= self.inh_spike_events[self.curr_inh_idx][0]:
            # Get the channel of the current spike event
            curr_channel = self.inh_spike_events[self.curr_inh_idx][1]

            # Check if the channel is valid (belongs to a channel in the down_channel_map therefore it has an output index)
            if curr_channel not in down_channel_map:
                self.curr_inh_idx += 1
                continue    # Skip the current spike event

            # Check if the next spike belongs to a channel that will already spike in this time step
            # If so, we don't add the event and stop looking for more events
            if curr_channel in spiking_channels:
                break

            # Add the channel to the list of spiking channels
            spiking_channels.add(curr_channel)

            # Get the output index of the current channel according to the down_channel_map
            out_idx = down_channel_map[curr_channel]
            if out_idx < self.s_out.shape[0]:   # Check if the channel is valid
                # It is not possible to send negative values or floats in the spike_data. The weight of the synapse should do the inhibition
                spike_data[out_idx] = 1.0   # Send spike (value corresponds to the punctual current of the spike event)

            # Move to the next spike event
            self.curr_inh_idx += 1


        if len(spiking_channels) > 0:   # Print the spike event if there are any spikes
            print(f"""Sending spike event at time: {currTime}({self.time_step}). Last (E/I) spike idx: {self.curr_exc_idx-1}/{self.curr_inh_idx-1}
                Spike times: {self.exc_spike_events[self.curr_exc_idx-1][0] if self.curr_exc_idx > 0 else "?"}/\
{self.inh_spike_events[self.curr_inh_idx-1][0] if self.curr_inh_idx > 0 else "?"}
                Spike_data: {spike_data}\n""")

        # Send spikes if self.curr_exc_idx > 0 else "?"
        # print("sending spike_data: ", spike_data, " at step: ", self.time_step)
        self.s_out.send(spike_data)

        # Stop the Process if there are no more spike events to send. (It will stop all the connected processes)
        # TODO: Should it be another process that stops the simulation? Such as the last LIF process
        # if self.curr_spike_idx >= 5: # len(self.spike_events):
        #    self.pause()

## Connect the Layers
To define the connectivity between the `SpikeGenerator` and the first `LIF` population, we use another `Dense` Layer.

In [24]:
# Create the Input Process
spike_event_gen = SpikeEventGen(out_shape=(n_spike_gen,),
                                exc_spike_events=up_spike_train_interval,
                                inh_spike_event=down_spike_train_interval,
                                name="SpikeEventsGenerator")

# If I connect the SpikeEventGen to the Dense Layer, the a_out value of the custom input will be rounded to 0 or 1 in the Dense Layer (it will not be a float) 
# However, setting the Dense weights to a float works instead
# Connect the SpikeEventGen to the Dense Layer
spike_event_gen.s_out.connect(dense_input.s_in)

# Connect the Dense_Input to the LIF1 Layer
dense_input.a_out.connect(configLIF.a_in)

### Take a look at the connections in the Input Layer

In [25]:
for proc in [spike_event_gen, dense_input, configLIF]:
    for port in proc.in_ports:
        print(f"Proc: {proc.name:<5} Port Name: {port.name:<5} Size: {port.size}")
    for port in proc.out_ports:
        print(f"Proc: {proc.name:<5} Port Name: {port.name:<5} Size: {port.size}")

Proc: SpikeEventsGenerator Port Name: s_out Size: 2
Proc: DenseInput Port Name: s_in  Size: 2
Proc: DenseInput Port Name: a_out Size: 256
Proc: lif1  Port Name: a_in  Size: 256
Proc: lif1  Port Name: s_out Size: 256


### Record Internal Vars over time
To record the evolution of the internal variables over time, we need a `Monitor`. For this example, we want to record the membrane potential of the `LIF` Layer, hence we need 1 `Monitors`.

We can define the `Var` that a `Monitor` should record, as well as the recording duration, using the `probe` function

In [26]:
from lava.proc.monitor.process import Monitor

monitor_lif1_v = Monitor()
monitor_lif1_u = Monitor()

# Connect the monitors to the variables we want to monitor
monitor_lif1_v.probe(configLIF.v, num_steps)
monitor_lif1_u.probe(configLIF.u, num_steps)  # Monitoring the net_current (u_exc + u_inh) of the LIF1 Process

## Execution
Now that we have defined the network, we can execute it. We will use the `run` function to execute the network.

### Run Configuration and Conditions

In [27]:
from lava.magma.core.run_conditions import RunContinuous, RunSteps
from lava.magma.core.run_configs import Loihi1SimCfg

# run_condition = RunContinuous()   # TODO: Change to this one
run_condition = RunSteps(num_steps=num_steps)
run_cfg = Loihi1SimCfg(select_tag="floating_pt")   # TODO: Check why we need this select_tag="floating_pt"

### Execute

In [28]:
configLIF.run(condition=run_condition, run_cfg=run_cfg)

Sending spike event at time: 1001(1). Last (E/I) spike idx: -1/0
                Spike times: ?/1000.9765625
                Spike_data: [0. 1.]

Sending spike event at time: 1002(2). Last (E/I) spike idx: 0/0
                Spike times: 1001.953125/1000.9765625
                Spike_data: [1. 0.]

Sending spike event at time: 1003(3). Last (E/I) spike idx: 1/0
                Spike times: 1002.44140625/1000.9765625
                Spike_data: [1. 0.]

Sending spike event at time: 1004(4). Last (E/I) spike idx: 1/1
                Spike times: 1002.44140625/1003.90625
                Spike_data: [0. 1.]

Sending spike event at time: 1006(6). Last (E/I) spike idx: 2/1
                Spike times: 1005.37109375/1003.90625
                Spike_data: [1. 0.]

Sending spike event at time: 1007(7). Last (E/I) spike idx: 3/1
                Spike times: 1005.859375/1003.90625
                Spike_data: [1. 0.]

Sending spike event at time: 1008(8). Last (E/I) spike idx: 3/2
               

### Retrieve recorded data

In [29]:
data_lif1_v = monitor_lif1_v.get_data()
data_lif1_u = monitor_lif1_u.get_data()

# print("Copying...")
data_lif1 = data_lif1_v.copy()
data_lif1["lif1"]["u"] = data_lif1_u["lif1"]["u"]   # Merge the dictionaries to contain both voltage and current


In [30]:
configLIF

In [31]:
# Check the shape to verify if it is printing the voltage for every step
print(len(data_lif1['lif1']['v']))     # Indeed, there are 300 values (same as the number of steps we ran the simulation for)

300


### Plot the recorded data

In [32]:
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt

# Boolean defining if we should use the monitor plot
MONITOR_PLOT = False

if MONITOR_PLOT:
    # Create a subplot for each monitored variable
    fig = plt.figure(figsize=(16, 10))
    ax0 = fig.add_subplot(221)
    ax0.set_title('Voltage (V) / time step')
    ax1 = fig.add_subplot(222)
    ax1.set_title('Current (U) / time step')


    # Plot the data
    monitor_lif1_v.plot(ax0, lif1.v)
    monitor_lif1_u.plot(ax1, lif1.u)


## Find the timesteps where the network spiked

In [33]:
from utils.data_analysis import find_spike_times

lif1_voltage_vals = np.array(data_lif1['lif1']['v'])
lif1_current_vals = np.array(data_lif1['lif1']['u'])
# preview_np_array(voltage_arr_1, "Voltage Array")

# Call the find_spike_times util function that detects the spikes in a voltage array
# TODO: Improve the find_spike_times method to view the current of the preview timestep to make sure it is a spike, instead of an inhibition
spike_times_lif1 = find_spike_times(lif1_voltage_vals, lif1_current_vals)

for (spike_time, neuron_idx) in spike_times_lif1:
    print(f"Spike time: {init_offset + spike_time * virtual_time_step_interval} (iter. {spike_time}) at neuron: {neuron_idx}")


Spike time: 1007 (iter. 7) at neuron: 3
Spike time: 1008 (iter. 8) at neuron: 4
Spike time: 1009 (iter. 9) at neuron: 0
Spike time: 1010 (iter. 10) at neuron: 3
Spike time: 1011 (iter. 11) at neuron: 17
Spike time: 1014 (iter. 14) at neuron: 55
Spike time: 1017 (iter. 17) at neuron: 1
Spike time: 1018 (iter. 18) at neuron: 48


## View the Voltage and Current dynamics with an interactive plot

Grab the data from the recorded variables

In [34]:
preview_np_array(lif1_voltage_vals, "Voltage Values", edge_items=3)

Voltage Values Shape: (300, 256).
Preview: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-3.00000000e-01 -3.00000000e-01 -3.00000000e-01 ... -3.00000000e-01
  -3.00000000e-01 -3.00000000e-01]
 [-5.85310293e-02 -1.86031510e-01  2.10000000e-02 ... -1.36632827e-01
   9.16731311e-03 -1.17585281e-01]
 ...
 [ 1.45765889e-09 -7.74566538e-10 -4.13922434e-11 ... -9.00866489e-10
   4.90928651e-10  8.27324005e-10]
 [ 1.35562277e-09 -7.20346880e-10 -3.84947863e-11 ... -8.37805835e-10
   4.56563645e-10  7.69411324e-10]
 [ 1.26072918e-09 -6.69922599e-10 -3.58001513e-11 ... -7.79159427e-10
   4.24604190e-10  7.15552532e-10]]


## Assemble the values to be plotted

In [35]:
from utils.line_plot import create_fig  # Import the function to create the figure
from bokeh.models import Range1d

# Define the x and y values
x = [val + init_offset for val in range(num_steps)]

v_y1 = [val[0] for val in lif1_voltage_vals]
v_y2 = [val[1] for val in lif1_voltage_vals]
v_y3 = [val[2] for val in lif1_voltage_vals]
v_y4 = [val[3] for val in lif1_voltage_vals]
v_y5 = [val[4] for val in lif1_voltage_vals]
v_y6 = [val[5] for val in lif1_voltage_vals]
v_y7 = [val[6] for val in lif1_voltage_vals]
v_y8 = [val[7] for val in lif1_voltage_vals]
v_y9 = [val[8] for val in lif1_voltage_vals]
v_y10 = [val[9] for val in lif1_voltage_vals]

# Create the plot
voltage_lif1_y_arrays = [
    (v_y1, "Neuron. 0"), (v_y2, "Neuron. 1"), (v_y3, "Neuron. 2"),
    (v_y4, "Neuron. 3"), (v_y5, "Neuron. 4"), # (v_y6, "Neuron. 5"),
    # (v_y7, "Neuron. 6"), (v_y8, "Neuron. 7"), (v_y9, "Neuron. 8"),
    # (v_y10, "Neuron. 9")
]    # List of tuples containing the y values and the legend label
# Define the box annotation parameters
box_annotation_voltage = {
    "bottom": 0,
    "top": v_th,
    "left": 0,
    "right": num_steps,
    "fill_alpha": 0.03,
    "fill_color": "green"
}

# Create the LIF1 Voltage
voltage_lif1_plot = create_fig(
    title="LIF1 Voltage dynamics", 
    x_axis_label='time (ms)', 
    y_axis_label='Voltage (V)',
    x=x, 
    y_arrays=voltage_lif1_y_arrays, 
    sizing_mode="stretch_both", 
    tools="pan, box_zoom, wheel_zoom, hover, undo, redo, zoom_in, zoom_out, reset, save",
    tooltips="Data point @x: @y",
    legend_location="top_right",
    legend_bg_fill_color="navy",
    legend_bg_fill_alpha=0.1,
    box_annotation_params=box_annotation_voltage,
    y_range=Range1d(-1.05, 1.05)
)


# Create the LIF1 Current
u_y1 = [val[0] for val in lif1_current_vals]
u_y2 = [val[1] for val in lif1_current_vals]
u_y3 = [val[2] for val in lif1_current_vals]
u_y4 = [val[3] for val in lif1_current_vals]
u_y5 = [val[4] for val in lif1_current_vals]
current_lif1_y_arrays = [(u_y1, "Neuron. 0"), (u_y2, "Neuron. 1"), (u_y3, "Neuron. 2"),
                          (u_y4, "Neuron. 3"), (u_y5, "Neuron. 4")]    # List of tuples containing the y values and the legend label
current_lif1_plot = create_fig(
    title="LIF1 Current dynamics", 
    x_axis_label='time (ms)', 
    y_axis_label='Current (U)',
    x=x, 
    y_arrays=current_lif1_y_arrays, 
    sizing_mode="stretch_both", 
    tools="pan, box_zoom, wheel_zoom, hover, undo, redo, zoom_in, zoom_out, reset, save",
    tooltips="Data point @x: @y",
    legend_location="top_right",
    legend_bg_fill_color="navy",
    legend_bg_fill_alpha=0.1,
    x_range=voltage_lif1_plot.x_range,    # Link the x-axis range to the voltage plot
)

# bplt.show(voltage_lif1_plot)

## Show the Plots assembled in a grid

In [36]:
import bokeh.plotting as bplt
from bokeh.layouts import gridplot

showPlot = True
if showPlot:
    # Create array of plots to be shown
    plots = [voltage_lif1_plot, current_lif1_plot]

    if len(plots) == 1:
        grid = plots[0]
    else:   # Create a grid layout
        grid = gridplot(plots, ncols=2, sizing_mode="stretch_both")

    # Show the plot
    bplt.show(grid)

## Export the plot to a file

In [37]:
export = False
OUTPUT_FOLDER = "./results/custom_subset_90-119_segment500_200"

if export:
    file_path = f"{OUTPUT_FOLDER}/{band_file_name}_output_0.07dv_5ch_.html"

    # Customize the output file settings
    bplt.output_file(filename=file_path, title="HFO Detection - Voltage and Current dynamics")

    # Save the plot
    bplt.save(grid)

## Export the Voltage and Current dynamics to a `.npy` file
In order to classify the feature neurons (Noisy, Silent, Ripple, or Fast Ripple Detector), we need to export the voltage and current dynamics to a `.npy` file to be analyzed by a Classification Algorithm

In [38]:
EXPORT_DYNAMICS = False
if EXPORT_DYNAMICS:
    # Define the file paths to save the Voltage and Current dynamics
    v_dynamics_file_path = f"{OUTPUT_FOLDER}/{band_file_name}_v_dynamics_0.07dv_5ch_time{init_offset}-{num_steps}-{virtual_time_step_interval}.npy"
    u_dynamic_file_path = f"{OUTPUT_FOLDER}/{band_file_name}_u_dynamics_0.07dv_5ch_time{init_offset}-{num_steps}-{virtual_time_step_interval}.npy"
    
    # Export the Voltage dynamics to a numpy file
    np.save(v_dynamics_file_path, lif1_voltage_vals)
    
    # Export the Current dynamics to a numpy file
    np.save(u_dynamic_file_path, lif1_current_vals)

## Export the Ground Truth data to a `.npy` file along with necessary Simulation Parameters

### Load the Ground Truth data from the `.npy` file

In [39]:
# Load the ground_truth data
ground_truth_file_name = f"{INPUT_PATH}/{band_file_name}_ground_truth.npy"

ground_truth = np.load(ground_truth_file_name)

preview_np_array(ground_truth, "ground_truth", edge_items=3)
print(f"Number of relevant events: {np.count_nonzero(ground_truth)}")

ground_truth Shape: (199,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Fast-Ripple',   3770.02, 0.) ... ('Fast-Ripple', 116096.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Fast-Ripple', 119000.  , 0.)]
Number of relevant events: 199


In [40]:
from utils.snn import SNNSimConfig

# Define the simulation configuration
snn_config = SNNSimConfig(ground_truth, init_offset, virtual_time_step_interval, num_steps)

snn_config_file_name = f"{OUTPUT_FOLDER}/{band_file_name}_snn_config.npy"
# Save the SNN Config Class to a npy file
np.save(snn_config_file_name, snn_config)

## Stop the Runtime

In [41]:
lif1.stop()